Connected to .venv (Python 3.12.6)

In [ ]:
suicide_rate_cleaned.head()

NameError: name 'suicide_rate_cleaned' is not defined

In [ ]:
import pandas as pd
import plotly.express as px
import itables 

# Load suicide_rate data
suicide_rate = pd.read_csv("data\suicide_rate_data.csv")

suicide_rate

# selecting applicable columns
suicide_rate_subset = suicide_rate [["Reference area", "Sex", "TIME_PERIOD", "OBS_VALUE"]]

suicide_rate_subset

# changing to representative column names
suicide_rate_subset = suicide_rate_subset.rename(columns={"Reference area": "country", "Sex": "sex", "TIME_PERIOD": "year", "OBS_VALUE": "mortality_count"})

suicide_rate_subset

# internet_use data
internet_use = pd.read_csv ("data\household_internet_use.csv")

internet_use 

# selecting applicable columns
internet_use_subset = internet_use[["Reference area", "TIME_PERIOD", "OBS_VALUE"]]

internet_use_subset

# changing to representative column names
internet_use_subset = internet_use_subset.rename(columns={"Reference area": "country", "TIME_PERIOD": "year", "OBS_VALUE": "percent_internet_household"})

internet_use_subset

# Load happiness_score data
happiness_data = pd.read_csv("data\hapiscore_whr.csv")
happiness_data

# changing dataset to a long format
happiness_data_long = happiness_data.melt(id_vars="country")
happiness_data_long

# changing to representative column names
happiness_data_long = happiness_data.melt(id_vars="country", var_name="year", value_name = "happiness_count").sort_values(["country", "year"])

happiness_data_long

# Function to check data types 

def check_data_types(df, name):
    print(f"Data Types for {name}:\n")
    print(df.dtypes)
    print("-" * 40)  # Separator for clarity

# checks data types for all three datasets
check_data_types(internet_use_subset, "Internet Use Subset")
check_data_types(suicide_rate_subset, "Suicide Rate Subset")
check_data_types(happiness_data_long, "Happiness Data Long")


# converting the 'year' column to integer type, and coercing errors to NaN
happiness_data_long['year'] = pd.to_numeric(happiness_data_long['year'], errors='coerce')

# Confirm the conversion
print(happiness_data_long['year'].dtype)


# filtering for year 2015 to 2022
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2015) & (happiness_data_long['year'] <= 2022)
]
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2015) & (happiness_data_long['year'] <= 2022)
]

# Check the result
happiness_data_long['year'].unique()

#calculating missing values for each dataset
happiness_missing = happiness_data_long.isna().sum()
internet_missing = internet_use_subset.isna().sum()
suicide_missing = suicide_rate_subset.isna().sum()


# combining the results into a single DataFrame
missing_summary = pd.DataFrame({
   'dataset': ['happiness_score', 'internet_use', 'suicide_rate'],
   'country': [happiness_missing['country'], internet_missing['country'], suicide_missing['country']],
   'year': [happiness_missing['year'], internet_missing['year'], suicide_missing['year']],
   'value_column': [happiness_missing['happiness_count'], internet_missing['percent_internet_household'], suicide_missing['mortality_count']]
})

missing_summary                      

# reshaping with melt and naming columns for clarity
missing_summary = missing_summary.melt(
    id_vars='dataset', var_name='dataset_columns', value_name='missing_count'
).sort_values('dataset')
missing_summary

# bar chart for missing values
fig = px.bar(
    missing_summary,
    x='dataset_columns',
    y='missing_count',
    color='dataset',
    title='Missing Values per Column Across Datasets',
    labels={'missing_count': 'Count of Missing Values', 'dataset_columns': 'Columns'},
    barmode='group'
)

fig.show()

# dropping missing value from dataset
happiness_data_long.dropna(inplace=True)


# function to check any persistent missing values
def check_missing_data(df, name):
    print(f"Missing Data Summary for {name}:")
    print(df.isna().sum())
    print("-" * 40)

# Check for missing values in each dataset
check_missing_data(happiness_data_long, "Happiness Data")
check_missing_data(internet_use_subset, "Internet Use Data")
check_missing_data(suicide_rate_subset, "Suicide Rate Data")


# rounding to 1 decimal for consistency with other datasets
internet_use_subset["percent_internet_household"] = internet_use_subset["percent_internet_household"].round(1)

internet_use_subset

# creating a pivoted version of the 'suicide_rate_subset' DataFrame 
# to analyze mortality rates by sex (Male, Female, Total)
# without modifying the original 'suicide_rate_subset
# pivoting the data so each sex becomes its own column
suicide_rate_pivot = suicide_rate_subset.pivot_table(
    index=['country', 'year'],
    columns='sex',
    values='mortality_count',
    aggfunc='sum'  # Aggregate if duplicates exist
).reset_index()

# renaming the columns for clarity
suicide_rate_pivot.columns.name = None  # removing column index name
suicide_rate_pivot = suicide_rate_pivot.rename(columns={
    'Male': 'mortality_male',
    'Female': 'mortality_female',
    'Total': 'mortality_total'
})

# cleaned dataset
suicide_rate_pivot.head()

# droping the 'sex' column from the suicide_rate_subset
suicide_rate_cleaned = suicide_rate_subset.drop(columns=['sex'])

suicide_rate_cleaned.head()

<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\h'
<>:36: SyntaxWarning: invalid escape sequence '\h'
<>:6: SyntaxWarning: invalid escape sequence '\s'
<>:21: SyntaxWarning: invalid escape sequence '\h'
<>:36: SyntaxWarning: invalid escape sequence '\h'
<ipython-input-2-13c58daca212>:6: SyntaxWarning: invalid escape sequence '\s'
  suicide_rate = pd.read_csv("data\suicide_rate_data.csv")
<ipython-input-2-13c58daca212>:21: SyntaxWarning: invalid escape sequence '\h'
  internet_use = pd.read_csv ("data\household_internet_use.csv")
<ipython-input-2-13c58daca212>:36: SyntaxWarning: invalid escape sequence '\h'
  happiness_data = pd.read_csv("data\hapiscore_whr.csv")


Data Types for Internet Use Subset:

country                        object
year                            int64
percent_internet_household    float64
dtype: object
----------------------------------------
Data Types for Suicide Rate Subset:

country             object
sex                 object
year                 int64
mortality_count    float64
dtype: object
----------------------------------------
Data Types for Happiness Data Long:

country             object
year                object
happiness_count    float64
dtype: object
----------------------------------------
int64


Missing Data Summary for Happiness Data:
country            0
year               0
happiness_count    0
dtype: int64
----------------------------------------
Missing Data Summary for Internet Use Data:
country                       0
year                          0
percent_internet_household    0
dtype: int64
----------------------------------------
Missing Data Summary for Suicide Rate Data:
country            0
sex                0
year               0
mortality_count    0
dtype: int64
----------------------------------------


,country,year,mortality_count
0,Ireland,2016,9.4
1,Iceland,2017,17.9
2,South Africa,2016,0.3
3,South Africa,2017,0.3
4,Austria,2015,5.9


In [ ]:
happy = set(happiness_data_long[]"country").unique())
sad = set(suicide_rate_cleaned["country"].unique())

SyntaxError: unmatched ')' (<ipython-input-3-d5932b185ff4>, line 1)

In [ ]:
happy = set(happiness_data_long["country"]).unique())
sad = set(suicide_rate_cleaned["country"].unique())

SyntaxError: unmatched ')' (<ipython-input-4-7e7418387339>, line 1)

In [ ]:
happy = set(happiness_data_long["country"].unique())
sad = set(suicide_rate_cleaned["country"].unique())

In [ ]:
import pandas as pd
import plotly.express as px
import itables 

# Load suicide_rate data
suicide_rate = pd.read_csv("data\suicide_rate_data.csv")

suicide_rate

# selecting applicable columns
suicide_rate_subset = suicide_rate [["Reference area", "Sex", "TIME_PERIOD", "OBS_VALUE"]]

suicide_rate_subset

# changing to representative column names
suicide_rate_subset = suicide_rate_subset.rename(columns={"Reference area": "country", "Sex": "sex", "TIME_PERIOD": "year", "OBS_VALUE": "mortality_count"})

suicide_rate_subset

# internet_use data
internet_use = pd.read_csv ("data\household_internet_use.csv")

internet_use 

# selecting applicable columns
internet_use_subset = internet_use[["Reference area", "TIME_PERIOD", "OBS_VALUE"]]

internet_use_subset

# changing to representative column names
internet_use_subset = internet_use_subset.rename(columns={"Reference area": "country", "TIME_PERIOD": "year", "OBS_VALUE": "percent_internet_household"})

internet_use_subset

# Load happiness_score data
happiness_data = pd.read_csv("data\hapiscore_whr.csv")
happiness_data

# changing dataset to a long format
happiness_data_long = happiness_data.melt(id_vars="country")
happiness_data_long

# changing to representative column names
happiness_data_long = happiness_data.melt(id_vars="country", var_name="year", value_name = "happiness_count").sort_values(["country", "year"])

happiness_data_long

# Function to check data types 

def check_data_types(df, name):
    print(f"Data Types for {name}:\n")
    print(df.dtypes)
    print("-" * 40)  # Separator for clarity

# checks data types for all three datasets
check_data_types(internet_use_subset, "Internet Use Subset")
check_data_types(suicide_rate_subset, "Suicide Rate Subset")
check_data_types(happiness_data_long, "Happiness Data Long")


# converting the 'year' column to integer type, and coercing errors to NaN
happiness_data_long['year'] = pd.to_numeric(happiness_data_long['year'], errors='coerce')

# Confirm the conversion
print(happiness_data_long['year'].dtype)


# filtering for year 2015 to 2022
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2015) & (happiness_data_long['year'] <= 2022)
]
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2015) & (happiness_data_long['year'] <= 2022)
]

# Check the result
happiness_data_long['year'].unique()

#calculating missing values for each dataset
happiness_missing = happiness_data_long.isna().sum()
internet_missing = internet_use_subset.isna().sum()
suicide_missing = suicide_rate_subset.isna().sum()


# combining the results into a single DataFrame
missing_summary = pd.DataFrame({
   'dataset': ['happiness_score', 'internet_use', 'suicide_rate'],
   'country': [happiness_missing['country'], internet_missing['country'], suicide_missing['country']],
   'year': [happiness_missing['year'], internet_missing['year'], suicide_missing['year']],
   'value_column': [happiness_missing['happiness_count'], internet_missing['percent_internet_household'], suicide_missing['mortality_count']]
})

missing_summary                      

# reshaping with melt and naming columns for clarity
missing_summary = missing_summary.melt(
    id_vars='dataset', var_name='dataset_columns', value_name='missing_count'
).sort_values('dataset')
missing_summary

# bar chart for missing values
fig = px.bar(
    missing_summary,
    x='dataset_columns',
    y='missing_count',
    color='dataset',
    title='Missing Values per Column Across Datasets',
    labels={'missing_count': 'Count of Missing Values', 'dataset_columns': 'Columns'},
    barmode='group'
)

fig.show()

# dropping missing value from dataset
happiness_data_long.dropna(inplace=True)


# function to check any persistent missing values
def check_missing_data(df, name):
    print(f"Missing Data Summary for {name}:")
    print(df.isna().sum())
    print("-" * 40)

# Check for missing values in each dataset
check_missing_data(happiness_data_long, "Happiness Data")
check_missing_data(internet_use_subset, "Internet Use Data")
check_missing_data(suicide_rate_subset, "Suicide Rate Data")


# rounding to 1 decimal for consistency with other datasets
internet_use_subset["percent_internet_household"] = internet_use_subset["percent_internet_household"].round(1)

internet_use_subset

# creating a pivoted version of the 'suicide_rate_subset' DataFrame 
# to analyze mortality rates by sex (Male, Female, Total)
# without modifying the original 'suicide_rate_subset
# pivoting the data so each sex becomes its own column
suicide_rate_pivot = suicide_rate_subset.pivot_table(
    index=['country', 'year'],
    columns='sex',
    values='mortality_count',
    aggfunc='sum'  # Aggregate if duplicates exist
).reset_index()

# renaming the columns for clarity
suicide_rate_pivot.columns.name = None  # removing column index name
suicide_rate_pivot = suicide_rate_pivot.rename(columns={
    'Male': 'mortality_male',
    'Female': 'mortality_female',
    'Total': 'mortality_total'
})

# cleaned dataset
suicide_rate_pivot.head()

# droping the 'sex' column from the suicide_rate_subset
suicide_rate_cleaned = suicide_rate_subset.drop(columns=['sex'])

suicide_rate_cleaned.head()

print(happiness_data_long.shape)
print(internet_use_subset.shape)
print(suicide_rate_cleaned.shape)
print(suicide_rate_pivot.shape)



happy = set(happiness_data_long["country"].unique())
sad = set(suicide_rate_cleaned["country"].unique())

Data Types for Internet Use Subset:

country                        object
year                            int64
percent_internet_household    float64
dtype: object
----------------------------------------
Data Types for Suicide Rate Subset:

country             object
sex                 object
year                 int64
mortality_count    float64
dtype: object
----------------------------------------
Data Types for Happiness Data Long:

country             object
year                object
happiness_count    float64
dtype: object
----------------------------------------
int64


<>:6: SyntaxWarning:

invalid escape sequence '\s'

<>:21: SyntaxWarning:

invalid escape sequence '\h'

<>:36: SyntaxWarning:

invalid escape sequence '\h'

<>:6: SyntaxWarning:

invalid escape sequence '\s'

<>:21: SyntaxWarning:

invalid escape sequence '\h'

<>:36: SyntaxWarning:

invalid escape sequence '\h'

<ipython-input-7-92e32146cf49>:6: SyntaxWarning:

invalid escape sequence '\s'

<ipython-input-7-92e32146cf49>:21: SyntaxWarning:

invalid escape sequence '\h'

<ipython-input-7-92e32146cf49>:36: SyntaxWarning:

invalid escape sequence '\h'



Missing Data Summary for Happiness Data:
country            0
year               0
happiness_count    0
dtype: int64
----------------------------------------
Missing Data Summary for Internet Use Data:
country                       0
year                          0
percent_internet_household    0
dtype: int64
----------------------------------------
Missing Data Summary for Suicide Rate Data:
country            0
sex                0
year               0
mortality_count    0
dtype: int64
----------------------------------------
(1092, 3)
(310, 3)
(888, 3)
(296, 5)


In [ ]:
happy = set(happiness_data_long["country"].unique())
sad = set(suicide_rate_cleaned["country"].unique())

In [ ]:
happy = set(happiness_data_long["country"].unique())
sad = set(suicide_rate_cleaned["country"].unique())

In [ ]:
happy - sad

{'Afghanistan',
 'Albania',
 'Algeria',
 'Armenia',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Belarus',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Central African Republic',
 'Chad',
 'China',
 'Comoros',
 'Congo, Dem. Rep.',
 'Congo, Rep.',
 "Cote d'Ivoire",
 'Cyprus',
 'Czech Republic',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eswatini',
 'Ethiopia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Ghana',
 'Guatemala',
 'Guinea',
 'Haiti',
 'Honduras',
 'Hong Kong, China',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Jamaica',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kosovo',
 'Kuwait',
 'Kyrgyz Republic',
 'Lao',
 'Lebanon',
 'Lesotho',
 'Liberia',
 'Libya',
 'Madagascar',
 'Malawi',
 'Malaysia',
 'Maldives',
 'Mali',
 'Malta',
 'Mauritania',
 'Mauritius',
 'Moldova',
 'Mongolia',
 'Montenegro',
 'Morocco',
 'Mozambique',
 'Myanmar',
 'Namibia',
 'Nepal',
 'Nicaragua',
 'Niger',
 'Nigeria',

In [ ]:
sad - happy 

{'Czechia', 'Korea', 'Türkiye', 'United Kingdom', 'United States'}

In [ ]:
happiness_data_long["country_code"]  = cc.convert(happiness_data_long["country"], to="ISO3")
internet_use_subset["country_code"]  = cc.convert(internet_use_subset["country"], to="ISO3")
suicide_rate_cleaned["country_code"] = cc.convert(suicide_rate_cleaned["country"], to="ISO3")

NameError: name 'cc' is not defined

In [ ]:
import country_converter as cc

In [ ]:
happiness_data_long["country_code"]  = cc.convert(happiness_data_long["country"], to="ISO3")
internet_use_subset["country_code"]  = cc.convert(internet_use_subset["country"], to="ISO3")
suicide_rate_cleaned["country_code"] = cc.convert(suicide_rate_cleaned["country"], to="ISO3")

UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (28 countries) not found in regex
European Union (27 countries) not found in r

In [ ]:
# Convert country names to ISO3 codes; invalid entries will become NaN
happiness_data_long["country_code"] = cc.convert(
    happiness_data_long["country"], to="ISO3", errors='coerce'
)
# Drop rows where 'country_code' is missing
happiness_data_long = happiness_data_long.dropna(subset=["country_code"])

# Verify the result
print(happiness_data_long.head())
print(f"New shape of the DataFrame: {happiness_data_long.shape}")

TypeError: CountryConverter.convert() got an unexpected keyword argument 'errors'

In [ ]:
# Convert country names to ISO3 codes; invalid entries will become NaN
happiness_data_long["country_code"] = cc.convert(
    happiness_data_long["country"], to="ISO3"
)
# Drop rows where 'country_code' is missing
happiness_data_long = happiness_data_long.dropna(subset=["country_code"])

# Verify the result
print(happiness_data_long.head())
print(f"New shape of the DataFrame: {happiness_data_long.shape}")

UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3


          country  year  happiness_count country_code
1640  Afghanistan  2015             39.8          AFG
1804  Afghanistan  2016             42.2          AFG
1968  Afghanistan  2017             26.6          AFG
2132  Afghanistan  2018             26.9          AFG
2296  Afghanistan  2019             23.8          AFG
New shape of the DataFrame: (1092, 4)


In [ ]:
internet_use_subset["country_code"]  = cc.convert(internet_use_subset["country"], to="ISO3")
# Drop rows where 'country_code' is missing
internet_use_subset = internet_use_subset.dropna(subset=["country_code"])

# Verify the result
print(internet_use_subset.head())
print(f"New shape of the DataFrame: {internet_use_subset.shape}")

OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (28 countries) not found in regex
European Union (27 countries) not found in regex


   country  year  percent_internet_household country_code
0   Mexico  2015                        39.2          MEX
1   Sweden  2020                        93.9          SWE
2   Sweden  2021                        93.2          SWE
3  Estonia  2017                        88.3          EST
4     OECD  2017                        84.2    not found
New shape of the DataFrame: (310, 4)


In [ ]:
internet_use_subset["country_code"]  = cc.convert(internet_use_subset["country"], to="ISO3")
# Drop rows where 'country_code' is missing
internet_use_subset = internet_use_subset.dropna(subset=["country_code"])

# Verify the result
print(internet_use_subset.head())
print(f"New shape of the DataFrame: {internet_use_subset.shape}")

OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
OECD not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (28 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (27 countries) not found in regex
European Union (28 countries) not found in regex
European Union (27 countries) not found in regex


   country  year  percent_internet_household country_code
0   Mexico  2015                        39.2          MEX
1   Sweden  2020                        93.9          SWE
2   Sweden  2021                        93.2          SWE
3  Estonia  2017                        88.3          EST
4     OECD  2017                        84.2    not found
New shape of the DataFrame: (310, 4)


In [ ]:
suicide_rate_cleaned["country_code"] = cc.convert(suicide_rate_cleaned["country"], to="ISO3")
# Drop rows where 'country_code' is missing
suicide_rate_cleaned = suicide_rate_cleaned.dropna(subset=["country_code"])

# Verify the result
print(suicide_rate_cleaned.head())
print(f"New shape of the DataFrame: {suicide_rate_cleaned.shape}")

        country  year  mortality_count country_code
0       Ireland  2016              9.4          IRL
1       Iceland  2017             17.9          ISL
2  South Africa  2016              0.3          ZAF
3  South Africa  2017              0.3          ZAF
4       Austria  2015              5.9          AUT
New shape of the DataFrame: (888, 4)


In [ ]:
import pandas as pd
import plotly.express as px
import itables 
import country_converter as cc

# Load suicide_rate data
suicide_rate = pd.read_csv("data\suicide_rate_data.csv")

suicide_rate

# selecting applicable columns
suicide_rate_subset = suicide_rate [["Reference area", "Sex", "TIME_PERIOD", "OBS_VALUE"]]

suicide_rate_subset

# changing to representative column names
suicide_rate_subset = suicide_rate_subset.rename(columns={"Reference area": "country", "Sex": "sex", "TIME_PERIOD": "year", "OBS_VALUE": "mortality_count"})

suicide_rate_subset

# internet_use data
internet_use = pd.read_csv ("data\household_internet_use.csv")

internet_use 

# selecting applicable columns
internet_use_subset = internet_use[["Reference area", "TIME_PERIOD", "OBS_VALUE"]]

internet_use_subset

# changing to representative column names
internet_use_subset = internet_use_subset.rename(columns={"Reference area": "country", "TIME_PERIOD": "year", "OBS_VALUE": "percent_internet_household"})

internet_use_subset

# Load happiness_score data
happiness_data = pd.read_csv("data\hapiscore_whr.csv")
happiness_data

# changing dataset to a long format
happiness_data_long = happiness_data.melt(id_vars="country")
happiness_data_long

# changing to representative column names
happiness_data_long = happiness_data.melt(id_vars="country", var_name="year", value_name = "happiness_count").sort_values(["country", "year"])

happiness_data_long

# Function to check data types 

def check_data_types(df, name):
    print(f"Data Types for {name}:\n")
    print(df.dtypes)
    print("-" * 40)  # Separator for clarity

# checks data types for all three datasets
check_data_types(internet_use_subset, "Internet Use Subset")
check_data_types(suicide_rate_subset, "Suicide Rate Subset")
check_data_types(happiness_data_long, "Happiness Data Long")


# converting the 'year' column to integer type, and coercing errors to NaN
happiness_data_long['year'] = pd.to_numeric(happiness_data_long['year'], errors='coerce')

# Confirm the conversion
print(happiness_data_long['year'].dtype)


# filtering for year 2015 to 2022
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2015) & (happiness_data_long['year'] <= 2022)
]
happiness_data_long = happiness_data_long[
    (happiness_data_long['year'] >= 2015) & (happiness_data_long['year'] <= 2022)
]

# Check the result
happiness_data_long['year'].unique()

#calculating missing values for each dataset
happiness_missing = happiness_data_long.isna().sum()
internet_missing = internet_use_subset.isna().sum()
suicide_missing = suicide_rate_subset.isna().sum()


# combining the results into a single DataFrame
missing_summary = pd.DataFrame({
   'dataset': ['happiness_score', 'internet_use', 'suicide_rate'],
   'country': [happiness_missing['country'], internet_missing['country'], suicide_missing['country']],
   'year': [happiness_missing['year'], internet_missing['year'], suicide_missing['year']],
   'value_column': [happiness_missing['happiness_count'], internet_missing['percent_internet_household'], suicide_missing['mortality_count']]
})

missing_summary                      

# reshaping with melt and naming columns for clarity
missing_summary = missing_summary.melt(
    id_vars='dataset', var_name='dataset_columns', value_name='missing_count'
).sort_values('dataset')
missing_summary

# bar chart for missing values
fig = px.bar(
    missing_summary,
    x='dataset_columns',
    y='missing_count',
    color='dataset',
    title='Missing Values per Column Across Datasets',
    labels={'missing_count': 'Count of Missing Values', 'dataset_columns': 'Columns'},
    barmode='group'
)

fig.show()

# dropping missing value from dataset
happiness_data_long.dropna(inplace=True)


# function to check any persistent missing values
def check_missing_data(df, name):
    print(f"Missing Data Summary for {name}:")
    print(df.isna().sum())
    print("-" * 40)

# Check for missing values in each dataset
check_missing_data(happiness_data_long, "Happiness Data")
check_missing_data(internet_use_subset, "Internet Use Data")
check_missing_data(suicide_rate_subset, "Suicide Rate Data")


# rounding to 1 decimal for consistency with other datasets
internet_use_subset["percent_internet_household"] = internet_use_subset["percent_internet_household"].round(1)

internet_use_subset

# creating a pivoted version of the 'suicide_rate_subset' DataFrame 
# to analyze mortality rates by sex (Male, Female, Total)
# without modifying the original 'suicide_rate_subset
# pivoting the data so each sex becomes its own column
suicide_rate_pivot = suicide_rate_subset.pivot_table(
    index=['country', 'year'],
    columns='sex',
    values='mortality_count',
    aggfunc='sum'  # Aggregate if duplicates exist
).reset_index()

# renaming the columns for clarity
suicide_rate_pivot.columns.name = None  # removing column index name
suicide_rate_pivot = suicide_rate_pivot.rename(columns={
    'Male': 'mortality_male',
    'Female': 'mortality_female',
    'Total': 'mortality_total'
})

# cleaned dataset
suicide_rate_pivot.head()

# droping the 'sex' column from the suicide_rate_subset
suicide_rate_cleaned = suicide_rate_subset.drop(columns=['sex'])

suicide_rate_cleaned.head()

Data Types for Internet Use Subset:

country                        object
year                            int64
percent_internet_household    float64
dtype: object
----------------------------------------
Data Types for Suicide Rate Subset:

country             object
sex                 object
year                 int64
mortality_count    float64
dtype: object
----------------------------------------
Data Types for Happiness Data Long:

country             object
year                object
happiness_count    float64
dtype: object
----------------------------------------
int64


<>:7: SyntaxWarning:

invalid escape sequence '\s'

<>:22: SyntaxWarning:

invalid escape sequence '\h'

<>:37: SyntaxWarning:

invalid escape sequence '\h'

<>:7: SyntaxWarning:

invalid escape sequence '\s'

<>:22: SyntaxWarning:

invalid escape sequence '\h'

<>:37: SyntaxWarning:

invalid escape sequence '\h'

<ipython-input-20-71c03c0be094>:7: SyntaxWarning:

invalid escape sequence '\s'

<ipython-input-20-71c03c0be094>:22: SyntaxWarning:

invalid escape sequence '\h'

<ipython-input-20-71c03c0be094>:37: SyntaxWarning:

invalid escape sequence '\h'



Missing Data Summary for Happiness Data:
country            0
year               0
happiness_count    0
dtype: int64
----------------------------------------
Missing Data Summary for Internet Use Data:
country                       0
year                          0
percent_internet_household    0
dtype: int64
----------------------------------------
Missing Data Summary for Suicide Rate Data:
country            0
sex                0
year               0
mortality_count    0
dtype: int64
----------------------------------------


,country,year,mortality_count
0,Ireland,2016,9.4
1,Iceland,2017,17.9
2,South Africa,2016,0.3
3,South Africa,2017,0.3
4,Austria,2015,5.9


In [ ]:
print(happiness_data_long.shape)
print(internet_use_subset.shape)
print(suicide_rate_cleaned.shape)
print(suicide_rate_pivot.shape)

(1092, 3)
(310, 3)
(888, 3)
(296, 5)


In [ ]:
# Step 1: Convert country names to ISO3 codes with 'errors=coerce' to handle unrecognized names
happiness_data_long["country_code"] = happiness_data_long["country"].apply(
    lambda x: cc.convert(x, to="ISO3", errors="coerce")
)

# Step 2: Drop rows where 'country_code' is NaN
happiness_data_long = happiness_data_long.dropna(subset=["country_code"])

# Step 3: Verify the updated DataFrame
print(happiness_data_long.head())
print(f"Shape after dropping invalid countries: {happiness_data_long.shape}")

TypeError: CountryConverter.convert() got an unexpected keyword argument 'errors'

In [ ]:
# Convert country names to ISO3 codes; invalid entries will become NaN
happiness_data_long["country_code"] = cc.convert(
    names=happiness_data_long["country"], to="ISO3", not_found=None
)
# Drop rows where 'country_code' is missing
happiness_data_long = happiness_data_long.dropna(subset=["country_code"])

# Verify the result
print(happiness_data_long.head())
print(f"New shape of the DataFrame: {happiness_data_long.shape}")

UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3


          country  year  happiness_count country_code
1640  Afghanistan  2015             39.8          AFG
1804  Afghanistan  2016             42.2          AFG
1968  Afghanistan  2017             26.6          AFG
2132  Afghanistan  2018             26.9          AFG
2296  Afghanistan  2019             23.8          AFG
New shape of the DataFrame: (1092, 4)


In [ ]:
# Convert country names to ISO3 codes; invalid entries will become NaN
happiness_data_long["country_code"] = cc.convert(
    names=happiness_data_long["country"], to="ISO3", not_found=None
)
# Step 2: Explicitly replace 'None' or invalid values with NaN
happiness_data_long["country_code"] = happiness_data_long["country_code"].replace([None, 'nan', ''], pd.NA)

# Drop rows where 'country_code' is missing
happiness_data_long = happiness_data_long.dropna(subset=["country_code"])

# Verify the result
print(happiness_data_long.head())
print(f"New shape of the DataFrame: {happiness_data_long.shape}")

UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3


          country  year  happiness_count country_code
1640  Afghanistan  2015             39.8          AFG
1804  Afghanistan  2016             42.2          AFG
1968  Afghanistan  2017             26.6          AFG
2132  Afghanistan  2018             26.9          AFG
2296  Afghanistan  2019             23.8          AFG
New shape of the DataFrame: (1092, 4)


In [ ]:
# Convert country names to ISO3 codes; invalid entries will become NaN
happiness_data_long["country_code"] = cc.convert(
    names=happiness_data_long["country"], to="ISO3", not_found=None
)
# Step 2: Explicitly replace 'None' or invalid values with NaN
happiness_data_long["country_code"] = happiness_data_long["country_code"].replace([None, 'nan', ''], pd.NA)

# Drop rows where 'country_code' is missing
happiness_data_long = happiness_data_long.dropna(subset=["country_code"])

# Verify the result
print(happiness_data_long.head())
print(f"Shape after dropping invalid countries: {happiness_data_long.shape}")

UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3
UAE not found in ISO3


          country  year  happiness_count country_code
1640  Afghanistan  2015             39.8          AFG
1804  Afghanistan  2016             42.2          AFG
1968  Afghanistan  2017             26.6          AFG
2132  Afghanistan  2018             26.9          AFG
2296  Afghanistan  2019             23.8          AFG
Shape after dropping invalid countries: (1092, 4)
